In [1]:
import cv2
import os

cam = cv2.VideoCapture(0)
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_id = input('\n enter user id end press <return> ==>  ')
count = 0
while(True):

    ret, img = cam.read()
    #img = cv2.flip(img, -1) # flip video image vertically
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1

        # Save the captured image into the datasets folder
        cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])

        cv2.imshow('image', img)

    k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
    #print(k)
    if k == 27:
        break
    elif count >= 30: # Take 30 face sample and stop video
         break

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()


 enter user id end press <return> ==>  7

 [INFO] Exiting Program and cleanup stuff


In [2]:
import cv2
import numpy as np
from PIL import Image
import os

# Path for face image database
path = 'dataset'
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");


In [3]:
# function to get the images and label data
def getImagesAndLabels(path):

    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]  
    #imagePaths = ["./dataset"]
    faceSamples=[]
    ids = []

    for imagePath in imagePaths:

        PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        img_numpy = np.array(PIL_img,'uint8')

        id = int(os.path.split(imagePath)[-1].split(".")[1])
        #print(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)

        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)

    return faceSamples,ids

print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml') # recognizer.save() worked on Mac, but not on Pi

# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))


 [INFO] Training faces. It will take a few seconds. Wait ...

 [INFO] 7 faces trained. Exiting Program


In [14]:
ak=0
em=0
mz=0
bg=0
sp=0
ms=0
b=0
total_classes=0
print(ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]


In [42]:
import cv2
import numpy as np
import os 
import time

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_SIMPLEX

#iniciate id counter
id = 0

# names related to ids: example ==> Marcelo: id=1,  etc
names = ['Abdul Kalam', 'Bill Gates', 'Elon Musk', 'Mark Zuckerberg', 'MS Dhoni', 'Sundar Pichai', 'benhar'] 

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
attendance=[]
intime=[]
while True:

    ret, img =cam.read()
    #img = cv2.flip(img, -1) # Flip vertically

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )

    for(x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        #print(id)
        #print(confidence)
        # Check if confidence is less them 100 ==> "0" is perfect match 
        if (confidence < 100):
            id2 = names[int(id-1)]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id2 = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(img, str(id2), (x+5,y-5), font, 1, (255,255,255), 2)
        #cv2.putText(img, names[int(id/30)], (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, names[int(id-1)], (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  

        if id2 not in attendance and id2!='unknown':
            attendance.append(id2)
            intime.append(time.asctime( time.localtime(time.time())))
            continue
    cv2.imshow('camera',img) 

    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
#print(attendance)
#print(intime)
# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()



 [INFO] Exiting Program and cleanup stuff


In [43]:
if 'Abdul Kalam' in attendance:
    ak=ak+1    
if 'Elon Musk' in attendance:
    em=em+1  
if 'Bill Gates' in attendance:
    bg=bg+1  
if 'Mark Zukerberg' in attendance:
    mz=mz+1  
if 'Sundar Pichai' in attendance:
    sp=sp+1  
if 'MS Dhoni' in attendance:
    ms=ms+1  
if 'benhar' in attendance:
    b=b+1
total_classes=total_classes+1    

In [44]:
final=[attendance,intime]
print(final)
import csv
with open("data10.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(final)

[['benhar', 'MS Dhoni', 'Bill Gates'], ['Wed Oct 21 10:39:59 2020', 'Wed Oct 21 10:40:02 2020', 'Wed Oct 21 10:40:05 2020']]


In [47]:
ea=[(ak/total_classes)*100,(em/total_classes)*100,(bg/total_classes)*100,(mz/total_classes)*100,(sp/total_classes)*100,(ms/total_classes)*100,(b/total_classes)*100]
print(ea)
exam=[names,ea]
print(exam)
import csv
with open("exam.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(exam)

[50.0, 50.0, 60.0, 0.0, 50.0, 90.0, 100.0]
[['Abdul Kalam', 'Bill Gates', 'Elon Musk', 'Mark Zuckerberg', 'MS Dhoni', 'Sundar Pichai', 'benhar'], [50.0, 50.0, 60.0, 0.0, 50.0, 90.0, 100.0]]


In [46]:
print(total_classes)

10


In [70]:
filename = "data2.csv"
# opening the file with w+ mode truncates the file
f = open(filename, "w+")
f.close()